**This notebook is an example process to develop ETL to create a pipeline that will feed Tableau with daily updates for player retention**

* Evaluate the data
* Create and optimize query
* Optimize table if I participate in the data engineering
* Calculate player retention rates (can also do daily and monthly active users)
* Save as parquet for data efficiency (considering millions of rows)
* Create Tableau .hyper file and load into Tableau server


In [ ]:
# First components would be completed in the AWS Redshift notebook environment
# Using camp grove as I saw it being promoted on the landing page 
# Evaluate the table structure
SELECT * 
FROM camp_grove_sessions 
LIMIT 10;



In [ ]:
# Update statistics for query optimization
ANALYZE camp_grove_sessions;

In [ ]:
# Check the execution plan before running the queyr
# If the query performs a full table scan, there may be missing indexes or improper sorting
EXPLAIN 
SELECT player_id, session_duration 
FROM camp_grove_sessions 
WHERE session_date >= CURRENT_DATE - INTERVAL '90 days';

In [ ]:
# Improve query performance by setting a distribution key and sort key for popular columns
# DISTKEY ensures even data distribution to improve join efficiency
# SORTKEY optimizes queries filtered by date ranges
ALTER TABLE camp_grove_sessions 
SET DISTSTYLE KEY 
DISTKEY (player_id) 
SORTKEY (session_date);

In [ ]:
# Vacuum reorganizes table storage for optimal query performance
VACUUM camp_grove_sessions;

In [ ]:
# Calculate player retention rates 
CREATE TABLE retention_rates AS
SELECT 
    install_date, 
    COUNT(DISTINCT player_id) AS new_players,
    COUNT(DISTINCT CASE WHEN session_date = install_date + INTERVAL '1 day' THEN player_id END) AS day_1_retention,
    COUNT(DISTINCT CASE WHEN session_date = install_date + INTERVAL '7 day' THEN player_id END) AS day_7_retention,
    COUNT(DISTINCT CASE WHEN session_date = install_date + INTERVAL '30 day' THEN player_id END) AS day_30_retention
FROM camp_grove_sessions
GROUP BY install_date;

In [ ]:
# Export this data to parquet files and store on S3
UNLOAD ('SELECT * FROM retention_rates')
TO 's3://your-bucket/camp_grove/retention_'
IAM_ROLE 'arn:aws:iam::your-account-id:role/your-redshift-role'
FORMAT AS PARQUET;

In [ ]:
# Assuming you have AWS Glue, this would be a serverless process to create a Tableau .hyper file using the parquet file
import boto3
import pandas as pd
from tableauhyperapi import HyperProcess, Connection, Telemetry, TableDefinition, SqlType, Inserter
from awsglue.context import GlueContext
from pyspark.sql import SparkSession

# Initialize Glue and Spark
spark = SparkSession.builder.appName("GlueToTableauHyper").getOrCreate()
glueContext = GlueContext(spark)

# Load Parquet from S3
s3_path = "s3://your-bucket/camp_grove/retention_"
df_spark = spark.read.parquet(s3_path)
df = df_spark.toPandas()  # Convert to Pandas

# Define Hyper File Path
hyper_file_path = "/tmp/retention_data.hyper"

# Define Hyper Table Schema
table_def = TableDefinition("retention_data", [
    ("install_date", SqlType.date()),
    ("new_players", SqlType.int()),
    ("day_1_retention", SqlType.int()),
    ("day_7_retention", SqlType.int()),
    ("day_30_retention", SqlType.int()),
])

# Convert to Hyper File
with HyperProcess(telemetry=Telemetry.DO_NOT_SEND_USAGE_DATA) as hyper:
    with Connection(endpoint=hyper.endpoint, database=hyper_file_path, create_mode="CREATE_AND_REPLACE") as connection:
        connection.catalog.create_table(table_def)
        with Inserter(connection, table_def) as inserter:
            inserter.add_rows(df.itertuples(index=False, name=None))
            inserter.execute()

print(f"Hyper file created: {hyper_file_path}")

# Upload to S3
s3_client = boto3.client("s3")
s3_client.upload_file(hyper_file_path, "your-bucket", "retention_data.hyper")


In [ ]:
# The last component would be to send this file to the server to update the visualization
import requests


# AWS S3 Configuration
S3_BUCKET_NAME = "your-s3-bucket"
S3_OBJECT_NAME = "retention_data.hyper"
LOCAL_HYPER_FILE = "/tmp/retention_data.hyper"  

# Download `.hyper` from S3
s3_client = boto3.client("s3")
s3_client.download_file(S3_BUCKET_NAME, S3_OBJECT_NAME, LOCAL_HYPER_FILE)
print(f"Downloaded {S3_OBJECT_NAME} from S3 to {LOCAL_HYPER_FILE}")

# Tableau Server Configuration
TABLEAU_SERVER_URL = "https://tableau.netflix.com"  # Or whatever your Tableau server is
TABLEAU_USERNAME = "service_username"
TABLEAU_PASSWORD = "service_password"
TABLEAU_SITE = ""  
PROJECT_NAME = "Game Analytics"  

# Authenticate with Tableau Server
auth_url = f"{TABLEAU_SERVER_URL}/api/3.10/auth/signin"
auth_payload = {
    "credentials": {
        "name": TABLEAU_USERNAME,
        "password": TABLEAU_PASSWORD,
        "site": {"contentUrl": TABLEAU_SITE}
    }
}
auth_response = requests.post(auth_url, json=auth_payload)
if auth_response.status_code == 200:
    auth_token = auth_response.json()["credentials"]["token"]
    site_id = auth_response.json()["credentials"]["site"]["id"]
    print("Authentication Successful")
else:
    print(f"Authentication Failed: {auth_response.text}")
    exit()

# Get Project ID
projects_url = f"{TABLEAU_SERVER_URL}/api/3.10/sites/{site_id}/projects"
headers = {"X-Tableau-Auth": auth_token}
projects_response = requests.get(projects_url, headers=headers)
projects = projects_response.json()["projects"]["project"]
project_id = next((p["id"] for p in projects if p["name"] == PROJECT_NAME), None)

if not project_id:
    print(f" Project '{PROJECT_NAME}' not found!")
    exit()

# Upload `.hyper` File
upload_url = f"{TABLEAU_SERVER_URL}/api/3.10/sites/{site_id}/datasources"
files = {"file": open(LOCAL_HYPER_FILE, "rb")}
upload_response = requests.post(upload_url, headers=headers, files=files)

if upload_response.status_code == 201:
    print("`.hyper` file successfully uploaded to Tableau Server")
else:
    print(f"Upload Failed: {upload_response.text}")

# Logout 
logout_url = f"{TABLEAU_SERVER_URL}/api/3.10/auth/signout"
requests.post(logout_url, headers=headers)
print("Logged out of Tableau Server")
